In [1]:
import pandas as pd
import sqlite3
import math
import munkres



In [2]:

conn = sqlite3.connect('D:\CSGOProGames\CSPAdb.sqlite3')
matches = pd.read_sql_query("""SELECT * FROM analyzer_MatchInfo""", conn)

playerdata = pd.read_sql_query("""SELECT *
                            FROM analyzer_PlayerData
                            JOIN analyzer_GameInfo 
                            ON analyzer_PlayerData.game_id = analyzer_GameInfo.id
                            WHERE played_map = 'inferno'
                            LIMIT 150000""", conn)

# Nades have different time stamps
nadedata = pd.read_sql_query("""SELECT * 
                        FROM analyzer_Nades
                        JOIN analyzer_GameInfo 
                        ON analyzer_Nades.game_id = analyzer_GameInfo.id
                        WHERE played_map = 'inferno'
                        LIMIT 150000""", conn)
conn.close()

In [3]:
fixNames = playerdata['team'].replace('Terrorist', 'T').replace('CounterTerrorist', 'CT')

In [4]:
playerdata['team'] = fixNames

In [5]:
playerdata.head()

,id,time,team,playernum,XPos,YPos,ZPos,weapon,game_id,id,played_map,t_score,ct_score,win_reason,match_id
0,376845,0.0,T,4,-1506.850,671.6946,-43.96875,Knife,217,217,inferno,0,0,TerroristWin,6
1,376846,0.0,T,5,-1655.827,418.4227,-63.34569,Knife,217,217,inferno,0,0,TerroristWin,6
2,376847,0.0,CT,6,2292.060,2027.6900,128.03130,Knife,217,217,inferno,0,0,TerroristWin,6
3,376848,0.0,CT,7,2363.130,1985.6600,128.03130,Knife,217,217,inferno,0,0,TerroristWin,6
4,376849,0.0,T,8,-1661.562,290.4704,-63.23479,Knife,217,217,inferno,0,0,TerroristWin,6


In [6]:
nadedata.head()

,id,time,XPos,YPos,ZPos,nade_type,game_id,id,played_map,t_score,ct_score,win_reason,match_id
0,3706,139.6,1192.6490,462.38760,125.80630,HE Nade,217,217,inferno,0,0,TerroristWin,6
1,3707,139.9,748.4474,-125.77450,631.44970,Flash Nade,217,217,inferno,0,0,TerroristWin,6
2,3708,141.6,786.7012,-83.28719,642.32220,Flash Nade,217,217,inferno,0,0,TerroristWin,6
3,3709,142.1,400.3172,2059.36300,130.88620,Decoy Nade,217,217,inferno,0,0,TerroristWin,6
4,3710,145.6,862.3829,-209.94330,97.66189,Decoy Nade,217,217,inferno,0,0,TerroristWin,6


In [21]:
bytick.describe()

TypeError: '>' not supported between instances of 'str' and 'int'

TypeError: '>' not supported between instances of 'str' and 'int'

In [7]:
bytick = playerdata.groupby(['game_id','time'])

In [46]:
players = [['CT', 'Dead'],['T', 'Any', 112, 114],['T', 'Dead']]
nades = []
threshold = 0.85
distancescale = 150.0
xoffset = 1760.0
yoffset = 790.0


matchinggames = []
matches = set()

# If user specified players
if players:
    
    for tick in bytick:
        tick = tick[1]
        if tick.iloc[0]['game_id'] in matches:
            continue
        similarities = [] 
        deadT = 5
        deadCT = 5
        for i,playerrow in tick.iterrows():
            tempscore = []

            if playerrow['team'] == 'T':
                deadT -= 1
            elif playerrow['team'] == 'CT':
                deadCT -= 1

            for player in players:
                score = 0.0          
                if player[0] == 'T' and player[1] == 'Dead':
                    tempscore.append('DT')
                    continue
                elif player[0] == 'CT' and player[1] == 'Dead':
                    tempscore.append('DCT')
                    continue

                if playerrow['team'] == player[0]:
                    if player[1] == playerrow['weapon'] or player[1] == 'Any':
                        score += 1
                    distance = math.sqrt(((playerrow['XPos']+xoffset)-(player[2]*9.75))**2 + ((playerrow['YPos']+yoffset)-(player[3]*9.75))**2)
                    score += 2**-(distance/distancescale)
                    tempscore.append(score)
                else:
                    tempscore.append(0)
            similarities.append(tempscore)
        # Consider continuing here if there aren't enough living players (probably low score)
        
        
        for i,row in enumerate(similarities):
            for j,column in enumerate(row):
                if column == 'DCT':
                    if deadCT > 0:
                        similarities[i][j] = 2 # Determine good value
                    else:
                        similarities[i][j] = 0
                elif column == 'DT':
                    if deadT > 0:
                        similarities[i][j] = 2 
                    else:
                        similarities[i][j] = 0
                # Turn profit function into cost function        
                similarities[i][j] = 2.0 - similarities[i][j]               

        # Assignment problem : https://en.wikipedia.org/wiki/Assignment_problem
        # Solved in O(n^3) using this algorithm
        # http://software.clapper.org/munkres/
        # This algorithm uses a cost calculation (i.e., minimization)
        # so we first need to subtract the values from the max value 
        # (performed during dead player value assignment)
        
        playersimilarity = 0
        if similarities:
            m = munkres.Munkres()
            indices = m.compute(similarities)
            for row,column in indices:
                # Revert to a similarity (high is better) and calculat total
                playersimilarity += 2.0 - similarities[row][column]
        
#         if playersimilarity < 0.7:
#             # If the players aren't correct, let's not even worry about the nades
#             continue
        
        if nades:
            similarities = []
            nadesimilarity = 0
            # Get all the nades from the same game within a defined time range.
            naderange = 2 
            naderows = nadedata.loc[(nadedata.time < playerrow.time + naderange) & (nadedata.time > playerrow.time - naderange) & (nadedata.game_id == playerrow.game_id)]
            if len(naderows.index):       
                for i,naderow in naderows.iterrows():
                    tempscore = []
                    for nade in nades:

                        score = 0.0       
                        if naderow['nade_type'] == nade[0]: 
                            # Customized for each map
                            distance = math.sqrt(((naderow['XPos']+xoffset)-(nade[1]*9.75))**2 + ((naderow['YPos']+yoffset)-(nade[2]*9.75))**2)
                            # Will need some kind of normalization
                            score += 2**-(distance/distancescale)
                            tempscore.append(score)
                        else:
                            tempscore.append(0)

                    similarities.append(tempscore)

                for i,row in enumerate(similarities):
                    for j,column in enumerate(row):
                        # Turn profit function into cost function
                        similarities[i][j] = 1 - similarities[i][j]

                if similarities:
                    m = munkres.Munkres()
                    indices = m.compute(similarities)
                    for row,column in indices:        
                        # Revert to a similarity (high is better) and calculat total
                        nadesimilarity += 1 - similarities[row][column]
        
        totalsimilarity = (float(playersimilarity) + float(nadesimilarity))/((2*len(players))+len(nades))  # Weighted normalization      
            
        if totalsimilarity > threshold:
            print(totalsimilarity, playersimilarity, nadesimilarity)
            matchinggames.append([totalsimilarity, playerrow['win_reason'], playerrow['match_id']])  # Use match id to get match link later
            matches.add(playerrow['game_id'])
            
print(matchinggames)
                

0.9397222617231563 5.638333570338938 0
0.9767098084406604 5.860258850643962 0
0.8585845463230538 5.151507277938323 0
0.8579249783676887 5.147549870206132 0
0.853413351619578 5.120480109717468 0
0.8569377236626554 5.1416263419759325 0
0.8671341669516491 5.202805001709895 0
0.8534926053310504 5.120955631986303 0
0.856661215345444 5.139967292072664 0
0.8716166733931647 5.229700040358988 0
0.850876243548457 5.105257461290742 0
[[0.9397222617231563, 'TerroristWin', 6], [0.9767098084406604, 'CTWin', 6], [0.8585845463230538, 'TerroristWin', 6], [0.8579249783676887, 'TerroristWin', 6], [0.853413351619578, 'TargetBombed', 6], [0.8569377236626554, 'CTWin', 7], [0.8671341669516491, 'TerroristWin', 7], [0.8534926053310504, 'TargetBombed', 8], [0.856661215345444, 'TargetBombed', 8], [0.8716166733931647, 'TerroristWin', 8], [0.850876243548457, 'Draw', 8]]


In [12]:
print(matchinggames)

[['CTWin', 222], ['CTWin', 222], ['CTWin', 222]]
